pip install qrisp 

https://www.qrisp.eu

DB-QITE implementation in Qrisp: https://qrisp.eu/reference/Algorithms/QITE.html

Experiments performed on QITE_experiments branch: https://github.com/eclipse-qrisp/Qrisp/tree/QITE_experiments with lowered simulator cutoff_ratio for high precision state vector simulations


In [ ]:
from qrisp import QuantumVariable
from qrisp.vqe import VQEProblem
from qrisp.vqe.problems.heisenberg import create_heisenberg_init_function

import networkx as nx
import numpy as np

import pickle
from datetime import datetime

from run_QITE import run_QITE

def save_data(data, filename):  
    # Open a file for writing
    with open(filename+'.pickle', 'wb') as file:
        # Write the object to the file
        pickle.dump(data, file)

def load_data(filename):
    with open(filename+'.pickle', 'rb') as file:
        # Load the object from the file
        data = pickle.load(file)
    return data

# Heisenberg XXX model

$$ H_{\text{TFIM}} = \sum_{j=0}^{L-1}(X_jX_{j+1}+Y_jY_{j+1}+Z_jZ_{j+1}) $$

In [ ]:
from qrisp.operators import X, Y, Z

L = 20
G = nx.Graph()
G.add_edges_from([(k, (k+1)%L) for k in range(L-1)]) 
# nx.draw(G, with_labels=True)

H = sum((X(i)*X(j)+Y(i)*Y(j)+Z(i)*Z(j)) for i,j in G.edges())
print(H)

# Define scaling factor
alpha = 10

# Hamiltonian simulation via second order Suzuki-Trotter formula with 2 steps
def exp_H(qv, t):
    H.trotterization(order=2, method='commuting')(qv, t/alpha, 2)

## DB-QITE with Singlet and HVA warm-start

In [ ]:
# Tensor product of singlet states
M = nx.maximal_matching(G)
U_singlet = create_heisenberg_init_function(M)

# HVA
H0 = sum((X(i)*X(j)+Y(i)*Y(j)+Z(i)*Z(j)) for i,j in M)
H1 = sum((X(i)*X(j)+Y(i)*Y(j)+Z(i)*Z(j)) for i,j in set(G.edges())-M)

def ansatz(qv, theta):
    # Rescaling t->1/4 equivalent to choosing rescaled inital parameters for HVA
    H1.trotterization(method='commuting')(qv, t=theta[1]/4)
    H0.trotterization(method='commuting')(qv, t=theta[0]/4)

HVA = VQEProblem(H, ansatz, 2, init_function=U_singlet)
U_HVA = HVA.train_function(QuantumVariable(G.number_of_nodes()), depth=1, max_iter=100)

# Calculate energy for HVA
def state_prep():
    qv = QuantumVariable(L)
    U_HVA(qv)
    return qv

print(H.expectation_value(state_prep)())

results = []

results.append(run_QITE(H, U_singlet, exp_H, np.linspace(.01,1.5,20), 5, method='GC', use_statevectors=True))
results.append(run_QITE(H, U_HVA, exp_H, np.linspace(.01,1.5,20), 5, method='GC', use_statevectors=True))

In [ ]:
# Save benchmark data
current_datetime = datetime.now()
date_time_string = current_datetime.strftime("%m-%d-%H")
#save_data(results,'data/benchmarks_2nd_L='+str(L)+'_'+date_time_string)